# Networks from OpenStreetMap


- crée un network à partir d'un fichier .osm et des trajectoires de véhiculess
- ajoute un flux de voiture sur les routes
- personnalise un Environnement pour le RL
- integrate it and run it in Flow.


In [1]:
from flow.core.params import VehicleParams
from flow.core.params import NetParams, SumoCarFollowingParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams
from flow.controllers import RLController, IDMController
from flow.networks import Network
from flow.core.params import InFlows

## Crée le network

Specifie les noms des routes du network dont les vehicules peuvent être s'insérer

In [2]:
EDGES_DISTRIBUTION = [
    "-100822066",
    "4794817",
    "4783299#0",
    "155558218",
]

créer la classe Network pour spécifier les itinéraires possibles

In [3]:
class IssyOSMNetwork(Network):

    def specify_routes(self, net_params):
        return {
            "-100822066": [ #N
                "-100822066",
                "-352962858#1",
                "-352962858#0",
                "-4786940#1",
                "-4786940#0",
            ],
            
            "4794817" : [ #Loop
                "4794817",
                "4786972#0",
                "4786972#1",
                "4786972#2",
                "4786965#1",
                "4786965#2",
                "4786965#3",
                "4795729",
                "-352962858#1",
                "4795742#0",
                "4795742#1",
                "4786965#3",
                "4786965#4",
                "4786965#5",
            ],
            
            "4783299#0": [    #E
                "4783299#0",
                "4783299#1",
                "4783299#2",
                "4783299#3",
                "4783299#4",
                "4783299#5",
                "4783299#6",
                "4786940#0",
                "4786940#1",
                "352962858#0",
                "352962858#1",
                "100822066",
            ],
            
            "155558218": [
                "155558218",
                "4786940#1",
                "352962858#0",
                "352962858#1",
                "100822066",
            ],     
        }

## Ajoute les flux de voiture

`IDMController` : The Intelligent Driver Model is a car-following model specifying vehicle dynamics by a differential equation for acceleration $\dot{v}$.

`RLController` : a trainable autuonomous vehicle whose actions are dictated by an RL agent. 

In [4]:
import MinicityRouter
from flow.controllers import ContinuousRouter

In [5]:
vehicles = VehicleParams()
vehicles.add("human",acceleration_controller=(IDMController, {}),num_vehicles=5)
vehicles.add("rl",acceleration_controller=(IDMController, {}),num_vehicles=10,color="cyan")

- `vehs_per_hour`: nombre de vehicule par heure, uniformément espacés. Par exemple, comme il y a $60 \times 60 = 3600$ secondes dans une heure, le parametre $\frac{3600}{5}=720$ va faire rentrer des vehicules dans le network toutes les $5$ secondes.

- `probability`: c'est la probabilité qu'un véhicule entre dans le network toutes les secondes. Par exemple, si on la fixe à $0.2$, alors chaque seconde de la simulation un véhicule aura $\frac{1}{5}$ chance d'entrer dans le network

- `period`: C'est le temps en secondes entre 2 véhicules qui sont insérés. Par exemple, le fixer à $5$ ferait rentrer des véhicules dans le network toutes les $5$ secondes (ce qui équivaut à mettre `vehs_per_hour` à $720$).

<font color='red'>
$\rightarrow$ Exactement 1 seul de ces 3 paramètres doit être configurer !
</font>

In [6]:
inflow = InFlows()
 
for edge in EDGES_DISTRIBUTION:
    inflow.add(veh_type = "human", 
               edge = edge,
               probability = 0.1,
               depart_lane = "best",
               name = "human",
               depart_speed = 7)

## Personnalise un Environnement pour le RL

plus de méthodes sur : http://berkeleyflow.readthedocs.io/en/latest/

In [7]:
# import the base environment class
from flow.envs import Env
from gym.spaces.box import Box
from gym.spaces import Tuple
from flow.core.experiment import Experiment
from myenv import myEnv
import numpy as np

## Lance une simulation

In [8]:
HORIZON = 3000

In [9]:
ADDITIONAL_ENV_PARAMS = {"max_accel": 2, "max_decel": 2}

# SUMO PARAM
sumo_params = SumoParams(sim_step=0.2, render=True) #, restart_instance=True)

# ENVIRONMENT PARAM
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=HORIZON)

# NETWORK PARAM
net_params = NetParams(inflows=inflow, osm_path='/home/lino/Documents/DQN_CIL4SYS/DQN_CIL4SYS/notebooks/issy.osm')

# NETWORK
network = IssyOSMNetwork

# INITIAL CONFIG
initial_config = InitialConfig(edges_distribution=EDGES_DISTRIBUTION) # spacing='random'

# dictionnaire FLOW_PARAM
flow_params = dict( exp_tag  = 'ISSY_test',
                    env_name = myEnv,  # using my new environment for the simulation
                    network  = network,
                    simulator= 'traci',
                    sim      = sumo_params,
                    env      = env_params,
                    net      = net_params,
                    veh      = vehicles,
                    initial  = initial_config)

# create EXPERIMENT with class created
exp = Experiment(flow_params)

# RUN SIMULATION SUMO
_ = exp.run(1)


**********************************************************
**********************************************************
**********************************************************
significantly decrease after large number of rollouts. In 
order to avoid this, set SumoParams(restart_instance=True).
**********************************************************
**********************************************************
**********************************************************
Round 0, return: 11857.41085705833
Average, std returns: 11857.41085705833, 0.0
Average, std velocities: 3.9524702856861196, 0.0
Average, std outflows: 1065.6, 0.0
Total time: 77.08390283584595
steps/second: 57.669291988802364
